In [ ]:
import pandas as pd
import numpy as np
import statistics as stat
import random as random
import math as math

# Ear Phenotypes

In [ ]:
useCols = [1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
df1 = pd.read_csv('../data/earphenotypesformatted.csv', na_values = {'a/n', 'a'}, usecols = useCols)
df2 = pd.read_csv('../data/earphenotypesformatted.csv', na_values = {'a/n', 'a'}, usecols = useCols)
fileList = [df1, df2]
traitList = ['Ear Width', 'Kernel Fill Length', 'Kernel Row Number', 'Kernels per Row', 'Ear Weight', 'Kernel Count', 'Cob Length', 'Cob Width', 'Cob Weight', '100 Kernel weight']

In [ ]:
# What if we just concat and pivot
ears = pd.concat([df1, df2])
ears = ears.reset_index(drop=False)
ears = ears.replace({'22.94.':'22.94'})
ears['Kernel Mass'] = ears['Ear Weight'].astype(float) - ears['Cob Weight'].astype(float)
ears['QR Code'].astype(str)
ears['QR Code'] = ears['QR Code'].str.upper()
ears['earNum'] = ears.groupby('QR Code').cumcount()
ears = ears[ears['earNum'] < 4]
plots = pd.pivot(ears, index = 'QR Code', columns = 'earNum')
traitList.append('Kernel Mass')
plots = plots.astype(float)

In [ ]:
plots.dtypes

In [ ]:
# Now that we have our nice data frame, we need to calculate the mean and standard error with all possible sets of 4 and all possible sets of 2 for each trait.
index30 = [1, 2, 3]
index31 = [0, 2, 3]
index32 = [0, 1, 2]
index20 = [0, 1]
index21 = [0, 2]
index22= [0, 3]
index23 = [1, 2]
index24= [1, 3]
index25= [2, 3]
index3List = [index30, index31, index32]
index3ListStr = ['30', '31', '32']
index2List = [index20, index21, index22, index23, index24, index25]
index2ListStr = ['20', '21', '22', '23', '24', '25']

results = pd.DataFrame()
for i in traitList:
    mean = i + 'mean' + '4'
    se = i + 'se' + '4'
    meanSeries = (plots[i][[0, 1, 2, 3]]).mean(axis = 1)
    results[mean] = meanSeries
    seSeries = (plots[i][[0, 1, 2, 3]]).sem(axis = 1)
    results[se] = seSeries
    
    for j in range(3):
        mean = i + 'mean' + index3ListStr[j]
        se = i + 'se' + index3ListStr[j]
        index = index3List[j]
        meanSeries = (plots[i][index]).mean(axis = 1)
        seSeries = (plots[i][index]).sem(axis = 1)
        results[mean] = meanSeries
        results[se] = seSeries
        
    for k in range(6):
        mean = i + 'mean' + index2ListStr[k]
        se = i + 'se' + index2ListStr[k]
        index = index2List[k]
        meanSeries = (plots[i][index]).mean(axis = 1)
        seSeries = (plots[i][index]).sem(axis = 1)
        results[mean] = meanSeries
        results[se] = seSeries

In [ ]:
meanSeries

In [ ]:
results

In [6]:
avgSE = pd.DataFrame()
for t in traitList:
    traitSE2 = t + '.SE.2'
    traitSE3 = t + '.SE.3'
    
    seCols2 = []
    for i in index2ListStr:
        colName = t + 'se' + i
        seCols2.append(colName)
     
    seCols3 = []
    for j in index3ListStr:
        colName = t + 'se' + j
        seCols3.append(colName)
        avgSE[traitSE2] = results[seCols2].mean(axis = 1)
        avgSE[traitSE3] = results[seCols3].mean(axis = 1)
    

In [ ]:
avgSE

In [7]:
corrPerSet = pd.DataFrame()
for t in traitList:
    fullMeanCol = t + 'mean' + '4'
    
    for i in index2ListStr:
        colName = t + 'mean' + i
        cor = results[colName].corr(results[fullMeanCol])
        corrPerSet[colName] = pd.Series()
        corrPerSet.at[0, colName] = cor
    for j in index3ListStr:
        colName = t + 'mean' + j
        cor = results[colName].corr(results[fullMeanCol])
        corrPerSet.at[0, colName] = cor
    

In [ ]:
corrPerSet.columns

In [8]:
summary = pd.DataFrame({'Set Size':[2, 3, 4]})

for t in traitList:
    sem = t + '.MeanSEM'
    corr = t + '.MeanCorr'
    sem2 = t + '.SE.2'
    sem3 = t + '.SE.3'
    semFull = t + 'se4'
    
    meanCols2 = []
    for i in index2ListStr:
        colName = t + 'mean' + i
        meanCols2.append(colName)
        
    meanCols3 = []
    for j in index3ListStr:
        colName = t + 'mean' + j
        meanCols3.append(colName)

    summary.at[0, sem] = avgSE[sem2].mean(axis = 0)
    summary.at[1, sem] = avgSE[sem3].mean(axis = 0)
    summary.at[2, sem] = results[semFull].mean(axis = 0)
    
    summary.at[0, corr] = corrPerSet[meanCols2].mean().values[0]
    summary.at[1, corr] = corrPerSet[meanCols3].mean().values[0]
    summary.at[2, corr] = 1

In [ ]:
# Export to csv
summary.to_csv('../analysis/RequiredNumberOfMeasurements.csv', index = False)

# NIR

In [27]:
nir = pd.read_excel('../data/NIRData.xlsx', sheet_name = 'in', dtype = {'Sample ID':str})
nir['protein'] = nir['Protein As is'] / (1 - (nir['Moisture'] / 100))
nir['oil'] = nir['Oil As is'] / ( 1 - (nir['Moisture'] / 100))
nir['fiber'] = nir['Fiber As is'] / (1 - (nir['Moisture'] / 100))
nir['ash'] = nir['Ash As is'] / (1 - (nir['Moisture'] / 100))
nir['starch'] = nir['Starch As is'] / (1 - (nir['Moisture'] / 100))
nir['qr'] = nir['Sample ID'].str.upper()
nir['qr'] = nir['qr'].astype(str)
nir = nir.rename(columns = {'Moisture':'moisture'})
traits = ['starch', 'protein', 'oil', 'fiber', 'ash', 'moisture']
nir = nir[['qr', 'starch', 'protein', 'oil', 'fiber', 'ash', 'moisture']]
nir = nir[~nir['qr'].str.contains('SYNGENTA')]

for t in traits:
    nir = nir[nir[t] >= 0]
nir['i'] = nir.groupby('qr').cumcount()
nir = nir[nir['i'] < 3]

nirWide = pd.pivot(nir, index = 'qr', columns = 'i')

In [28]:
index3 = [0, 1, 2]
index20 = [1, 2]
index21 = [0, 2]
index22 = [0, 1]
nirResults = pd.DataFrame()

for t in traits:
    mean3 = t + '.Mean.3'
    se3 = t + '.SE.3'
    meanSeries = 


starch             \
i                                                           0          1   
qr                                                                         
4101$ROW2$RANGE2$PHN46 X W606S                      71.538868        NaN   
4102$ROW2$RANGE3$PHK56 X W606S                      67.527675  67.677406   
4103$ROW2$RANGE4$PHK56 X PHJ89                      68.685594  69.597725   
4104$ROW2$RANGE5$PHJ40 X LH82                       67.606675  67.697161   
4105$ROW2$RANGE6$PHP02 X PHK76                      66.754009  67.296390   
...                                                       ...        ...   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...  71.570772  71.494152   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...  72.430525  72.369726   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...  70.271168  70.255616   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...  70.875069  70.957787   
nan                                                 69.733220        NaN   

                                                                 protein  \
i                                                           2          0   
qr                                                                         
4101$ROW2$RANGE2$PHN46 X W606S                            NaN   9.011105   
4102$ROW2$RANGE3$PHK56 X W606S                      67.497097  10.437533   
4103$ROW2$RANGE4$PHK56 X PHJ89                      69.796821   9.253417   
4104$ROW2$RANGE5$PHJ40 X LH82                       70.799537  10.350655   
4105$ROW2$RANGE6$PHP02 X PHK76                      67.259042  10.523006   
...                                                       ...        ...   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...  70.885246   8.750820   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...  72.529775   7.090869   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...  70.005522   7.393470   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...  70.665795   8.013209   
nan                                                       NaN   9.549015   

                                                                          \
i                                                           1          2   
qr                                                                         
4101$ROW2$RANGE2$PHN46 X W606S                            NaN        NaN   
4102$ROW2$RANGE3$PHK56 X W606S                      11.097073  10.946902   
4103$ROW2$RANGE4$PHK56 X PHJ89                       9.446083   9.674703   
4104$ROW2$RANGE5$PHJ40 X LH82                       10.620400  10.070578   
4105$ROW2$RANGE6$PHP02 X PHK76                       9.519312   9.293331   
...                                                       ...        ...   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...   8.875287   9.234973   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...   7.039771   6.969563   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...   7.458227   7.598012   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...   8.067894   8.020050   
nan                                                       NaN        NaN   

                                                         oil            \
i                                                          0         1   
qr                                                                       
4101$ROW2$RANGE2$PHN46 X W606S                      4.717081       NaN   
4102$ROW2$RANGE3$PHK56 X W606S                      3.879810  3.863971   
4103$ROW2$RANGE4$PHK56 X PHJ89                      4.300736  4.401853   
4104$ROW2$RANGE5$PHJ40 X LH82                       4.309252  3.985279   
4105$ROW2$RANGE6$PHP02 X PHK76                      3.343465  3.662888   
...                                                      ...       ...   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...  3.226865  3.049514   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$...  4.267755  4.208439   
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2